In [2]:
# Imports - avoid importing multiple copies
import pandas as pd
import sqlite3
from decimal import Decimal
import locale

In [3]:
dfcsv = pd.read_csv("../Data/EPA_SmartLocationDatabase_V3_Jan_2021_Final.csv", dtype_backend="pyarrow")

print(dfcsv.dtypes)

display(dfcsv)

OBJECTID         int64[pyarrow]
GEOID10         double[pyarrow]
GEOID20         double[pyarrow]
STATEFP          int64[pyarrow]
COUNTYFP         int64[pyarrow]
                     ...       
D3B_Ranked       int64[pyarrow]
D4A_Ranked       int64[pyarrow]
NatWalkInd      double[pyarrow]
Shape_Length    double[pyarrow]
Shape_Area      double[pyarrow]
Length: 117, dtype: object


,OBJECTID,GEOID10,GEOID20,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,CSA,CSA_Name,CBSA,...,D5DRI,D5DE,D5DEI,D2A_Ranked,D2B_Ranked,D3B_Ranked,D4A_Ranked,NatWalkInd,Shape_Length,Shape_Area
0,1,481130000000.0,481130000000.0,48,113,7825,4,206,"Dallas-Fort Worth, TX-OK",19100,...,0.184697,0.000476,0.137707,6,14,15,17,14.0,3110.36082,297836.0831
1,2,481130000000.0,481130000000.0,48,113,7825,2,206,"Dallas-Fort Worth, TX-OK",19100,...,0.323221,0.000801,0.231868,3,10,12,14,10.833333,3519.46911,484945.1466
2,3,481130000000.0,481130000000.0,48,113,7825,3,206,"Dallas-Fort Worth, TX-OK",19100,...,0.314628,0.000736,0.213146,1,1,7,17,8.333333,1697.091802,106705.9281
3,4,481130000000.0,481130000000.0,48,113,7824,1,206,"Dallas-Fort Worth, TX-OK",19100,...,0.229821,0.000708,0.205018,16,10,17,17,15.666667,2922.609204,481828.4303
4,5,481130000000.0,481130000000.0,48,113,7824,2,206,"Dallas-Fort Worth, TX-OK",19100,...,0.164863,0.000433,0.125296,4,7,11,14,10.166667,3731.971773,687684.7752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220735,220736,780310000000.0,780310000000.0,78,30,961000,2,<NA>,<NA>,<NA>,...,-99999.0,-99999.0,-99999.0,1,1,20,1,7.333333,3414.446949,335585.6791
220736,220737,780310000000.0,780310000000.0,78,30,961000,3,<NA>,<NA>,<NA>,...,-99999.0,-99999.0,-99999.0,1,1,20,1,7.333333,2421.025608,292430.4642
220737,220738,780310000000.0,780310000000.0,78,30,961000,5,<NA>,<NA>,<NA>,...,-99999.0,-99999.0,-99999.0,1,1,20,1,7.333333,1955.909418,161939.4821
220738,220739,780310000000.0,780310000000.0,78,30,960700,3,<NA>,<NA>,<NA>,...,-99999.0,-99999.0,-99999.0,1,1,10,1,4.0,16896.76887,10389656.11


In [7]:
dfexcel = pd.read_excel("../Data/6.-Hospital-Data-with-Mixed-Numbers-and-Characters.xlsx", 1, dtype_backend="pyarrow")

print(dfexcel.dtypes)

display(dfexcel)

Description                        string[pyarrow]
Basic Unit                         string[pyarrow]
Beginning Balance                  double[pyarrow]
Quantity Received                   int64[pyarrow]
Quantity Dispensed                 double[pyarrow]
Losses and Adjustments              int64[pyarrow]
Ending Balance (Physical Count)    double[pyarrow]
dtype: object


,Description,Basic Unit,Beginning Balance,Quantity Received,Quantity Dispensed,Losses and Adjustments,Ending Balance (Physical Count)
0,Tab. Cefuroxime 250mg,10,1000.0,0,0.0,0,0.0
1,Tab. Cefuroxime 250mg,10,1000.0,0,0.0,0,0.0
2,Tab. Cefuroxime 250mg,10,1000.0,0,0.0,0,0.0
3,Cap Ampicillin,100,350.0,0,0.0,0,0.0
4,Cap Ampicillin,100,350.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...
5843,Insulin (Insulutard),Vial,0.0,2,0.0,0,2.0
5844,Inj Amoxicillin 500mg,vials,54.0,100,79.0,0,75.0
5845,Inj Amoxy/clav 1.2,vials,7.0,0,7.0,0,0.0
5846,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [9]:
conn = sqlite3.connect("../Data/example.db")

query = "SELECT * FROM Students WHERE Grade BETWEEN 80 AND 90"

dfsql = pd.read_sql(query, conn, dtype_backend="pyarrow")

print(dfsql.dtypes)

display(dfsql)

conn.close()

ID         int64[pyarrow]
Name      string[pyarrow]
Grade      int64[pyarrow]
Age        int64[pyarrow]
Gender    string[pyarrow]
City      string[pyarrow]
dtype: object


,ID,Name,Grade,Age,Gender,City
0,1,Alice Johnson,85,20,Female,New York
1,5,Emma Taylor,89,22,Female,Phoenix
2,7,Grace Anderson,82,21,Female,San Antonio
3,9,Isabel Robinson,90,22,Female,Dallas
4,10,Jack Clark,83,20,Male,San Jose
5,11,Katie Lee,88,21,Female,Austin
6,14,Noah Hall,84,20,Male,San Francisco
7,17,Quinn Baker,80,22,Female,Charlotte
8,19,Samuel Ramirez,87,21,Male,El Paso
9,21,Uma Phillips,86,20,Female,Boston


In [24]:
# Create pandas dataframe from sales file
dfSales = pd.read_csv("https://raw.githubusercontent.com/BriDeWaltCCC/PFDADataSets/main/sales.csv", dtype_backend="pyarrow")

# print(dfSales.dtypes)
# display(dfSales)
print(len(dfSales), " total sales in input file")
print(len(dfSales[dfSales["price"] > 50.0]), " sales of products over $50")
print(len(dfSales[dfSales["price"] > 75.0]), " sales of products over $75")

# more ways to do this - https://pythonguides.com/count-rows-in-pandas-dataframe/

dfSales.to_sql()

1000  total sales in input file
512  sales of products over $50
283  sales of products over $75
